# Regression: Predicting Pipe Pressure Drop

This notebook demonstrates supervised regression applied to a **CFD surrogate modeling** problem:
predicting fluid pressure drop across a pipe from flow parameters (flow rate, density, viscosity, length, diameter).
We generate synthetic data, fit a linear regression model, and visualize predictions vs. ground truth.

## Physics Problem: Pressure Drop in a Pipe

### Problem Illustration

```
                    Inlet                              Outlet
                     P₁                                   P₂
                     │                                    │
        ┌────────────▼────────────────────────────────────▼─────────┐
        │                                                             │
        │  ╔═══════════════════════════════════════════════════════╗ │
        │  ║        Pipe with flowing fluid                       ║ │
        │  ║  Q: flow rate (m³/s)                                 ║ │
        │  ║  ρ: density (kg/m³)                                  ║ │
        │  ║  μ: viscosity (Pa·s)                                 ║ │
        │  ║  L: pipe length (m)                                  ║ │
        │  ║  D: pipe diameter (m)                                ║ │
        │  ╚═══════════════════════════════════════════════════════╝ │
        │                                                             │
        └─────────────────────────────────────────────────────────────┘

              Δp = P₁ - P₂ (pressure drop)
```

### Physics Concept
When fluid flows through a pipe, friction and inertial forces cause a **pressure drop** (Δp) between inlet and outlet. This is governed by the **Darcy–Weisbach equation**:

$$\Delta p = \frac{1}{2} \rho v^2 \frac{L}{D} + \text{(viscous corrections)}$$

Where:
- **ρ**: fluid density (affects pressure magnitude)
- **v**: flow velocity proportional to Q/A (kinetic energy term)
- **L/D**: friction factor proxy (longer pipes, smaller diameters → higher loss)
- **μ**: viscosity (damping effect on flow)

### Machine Learning Goal
Given measurements of **Q, ρ, μ, L, D** (features), predict **Δp** (target) using linear regression.
The model learns the implicit coefficients that relate these features to pressure drop.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
warnings.filterwarnings('ignore')

try:
    import xgboost as xgb
    HAS_XGBOOST = True
except ImportError:
    HAS_XGBOOST = False

plt.rcParams['figure.figsize'] = (10, 6)

In [ ]:
def compute_pressure_drop(flow_rate, density, viscosity, length, diameter):
    area = np.pi * (diameter / 2) ** 2
    velocity = flow_rate / area
    dp = 0.5 * density * velocity ** 2 * (length / diameter)
    dp += viscosity * flow_rate / (diameter ** 2)
    return dp

rng = np.random.RandomState(0)
n = 500
flow_rate = rng.uniform(0.01, 0.5, size=n)
density = rng.uniform(950.0, 1050.0, size=n)
viscosity = rng.uniform(1e-4, 5e-3, size=n)
length = rng.uniform(0.5, 10.0, size=n)
diameter = rng.uniform(0.01, 0.2, size=n)
X = np.vstack([flow_rate, density, viscosity, length, diameter]).T
y = np.array([compute_pressure_drop(q, rho, mu, L, D) for q, rho, mu, L, D in X])
y += rng.normal(scale=0.05 * np.mean(y), size=y.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Dictionary to store models and results
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=0, n_jobs=-1),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=0),
}

# Add XGBoost if available
if HAS_XGBOOST:
    models['XGBoost'] = xgb.XGBRegressor(n_estimators=100, random_state=0, verbosity=0)

# Train and evaluate all models
results = []
predictions = {}

for name, model in models.items():
    # Train
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    predictions[name] = y_pred
    
    # Compute metrics
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        'Model': name,
        'MSE': mse,
        'MAE': mae,
        'RMSE': rmse,
        'R²': r2
    })

# Create results DataFrame
results_df = pd.DataFrame(results)
print("=" * 80)
print("Model Comparison Metrics")
print("=" * 80)
print(results_df.to_string(index=False))
print("=" * 80)

In [ ]:
## Predicted vs True for all models
n_models = len(predictions)
fig, axes = plt.subplots(1, n_models, figsize=(5*n_models, 4))
if n_models == 1:
    axes = [axes]

for idx, (name, y_pred) in enumerate(predictions.items()):
    ax = axes[idx]
    ax.scatter(y_test, y_pred, alpha=0.6, s=30)
    ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
    ax.set_xlabel('True pressure drop', fontsize=10)
    ax.set_ylabel('Predicted pressure drop', fontsize=10)
    r2 = r2_score(y_test, y_pred)
    ax.set_title(f'{name}\n$R^2$ = {r2:.3f}', fontsize=11)
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
## Metrics comparison bar plots
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

metrics = ['MSE', 'RMSE', 'MAE', 'R²']
axes_flat = axes.flatten()

for idx, metric in enumerate(metrics):
    ax = axes_flat[idx]
    values = results_df[metric].values
    colors = plt.cm.viridis(np.linspace(0, 1, len(results_df)))
    
    bars = ax.bar(results_df['Model'], values, color=colors, alpha=0.7, edgecolor='black')
    ax.set_ylabel(metric, fontsize=11)
    ax.set_title(f'{metric} Comparison', fontsize=12, fontweight='bold')
    ax.grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.2e}' if metric != 'R²' else f'{height:.3f}',
                ha='center', va='bottom', fontsize=9)
    
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')

plt.tight_layout()
plt.show()

## Model Interpretation and Insights

### Key Findings:

1. **R² Score**: Indicates how well the model explains variance in the data
   - Higher is better (max 1.0)
   - Measures proportion of target variability captured by the model

2. **RMSE (Root Mean Squared Error)**: 
   - In same units as target (pressure drop)
   - Penalizes large errors more than small ones
   - Useful when outliers matter

3. **MAE (Mean Absolute Error)**:
   - Average absolute deviation from true values
   - More robust to outliers than RMSE
   - Easier to interpret than MSE

4. **Best Model Selection**:
   - Highest R² indicates best fit
   - Compare RMSE/MAE for practical error understanding
   - Tree-based models often capture non-linearity better than linear regression
